In [51]:
# ═════════════════════════════════════════════════════════════════════════════
# RIFE 50 FPS UPSAMPLING PIPELINE FOR EQUINE PAIN DETECTION
# ═════════════════════════════════════════════════════════════════════════════
# PURPOSE: Convert 7–10 FPS cropped facial regions → smooth 50 FPS interpolation
# REGIONS: chin, eye_left, eye_right per equine subject
# KEY FEATURE: RIFE inserts frames between existing frames (2*N - 1 output frames)
# ═════════════════════════════════════════════════════════════════════════════

from pathlib import Path
import subprocess
import os
import sys

In [52]:
# ═════════════════════════════════════════════════════════════════════════════
# STEP 1: CONFIGURE PATHS
# ═════════════════════════════════════════════════════════════════════════════
# This cell sets up all required paths. Update paths if your setup differs.

BASE = Path(r"D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE")

# Input: Balanced crops (7-10 FPS, from previous preprocessing step)
CROPS_ROOT = BASE / "dataset" / "cropped_regions_BALANCED"

# RIFE installation folder (with inference_video.py)
RIFE_ROOT = Path(r"D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\Practical-RIFE")

# Python executable for RIFE environment (isolated venv to avoid conflicts)
RIFE_PY = r"D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\rife_env\Scripts\python.exe"

# Output: 50 FPS upsampled crops (created if not exists)
OUTPUT_50FPS = BASE / "dataset" / "cropped_regions_50FPS"
OUTPUT_50FPS.mkdir(exist_ok=True)

# ffmpeg binary for image/video conversion
FFMPEG_BIN = str(BASE / "ffmpeg" / "bin" / "ffmpeg.exe")

In [53]:
# ═════════════════════════════════════════════════════════════════════════════
# STEP 2: VERIFY ENVIRONMENT & PATHS
# ═════════════════════════════════════════════════════════════════════════════
# Checks that all required components exist before proceeding.

if not (RIFE_ROOT / "inference_video.py").exists():
    print("ERROR: RIFE not found!")
    print(f"Expected inference_video.py at: {RIFE_ROOT / 'inference_video.py'}")
    print("Solution: Download ECCV2022-RIFE and extract to Practical-RIFE folder")
    sys.exit()

if not Path(RIFE_PY).exists():
    print("ERROR: RIFE Python environment not found!")
    print(f"Expected python.exe at: {RIFE_PY}")
    print("Solution: Create venv with: python -m venv rife_env")
    sys.exit()

if not CROPS_ROOT.exists():
    print("ERROR: Input crops not found!")
    print(f"Expected cropped regions at: {CROPS_ROOT}")
    print("Solution: Run balanced_croppings.ipynb first")
    sys.exit()

if not Path(FFMPEG_BIN).exists():
    print("ERROR: ffmpeg not found!")
    print(f"Expected ffmpeg.exe at: {FFMPEG_BIN}")
    print("Solution: Download ffmpeg and extract to ffmpeg/ folder")
    sys.exit()

print("✓ All components verified!")
print(f"  Input:  {CROPS_ROOT}")
print(f"  Output: {OUTPUT_50FPS}")
print(f"  RIFE:   {RIFE_ROOT}")
print(f"  ffmpeg: {FFMPEG_BIN}")

✓ All components verified!
  Input:  D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\dataset\cropped_regions_BALANCED
  Output: D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\dataset\cropped_regions_50FPS
  RIFE:   D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\Practical-RIFE
  ffmpeg: D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\ffmpeg\bin\ffmpeg.exe


In [ ]:
# ═════════════════════════════════════════════════════════════════════════════
# FINAL TEST — RUN THIS NOW (WILL WORK)
# [DEPRECATED — Diagnostic cell, not needed for main pipeline]
# ═════════════════════════════════════════════════════════════════════════════
# import os
# import subprocess
# from pathlib import Path
# import sys

# BASE = Path(r"D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE")
# RIFE_DIR = BASE / "Practical-RIFE"
# RIFE_PY = BASE / "rife_env" / "Scripts" / "python.exe"

# os.chdir(RIFE_DIR)

# # Check what images actually exist
# chin_folder = BASE / "dataset" / "cropped_regions_BALANCED" / "S1_Video" / "chin"
# images = sorted(list(chin_folder.glob("*.jpg")))
# print(f"Found {len(images)} images")
# if images:
#     print(f"First image: {images[0].name}")
#     print(f"Last image: {images[-1].name}")
#     
#     # Extract starting number from first image
#     first_num = int(images[0].name.replace(".jpg", ""))
#     print(f"Starting number: {first_num}")

# if len(images) == 0:
#     print("ERROR: No images found!")
# else:
#     # Determine the correct numbering format
#     first_name = images[0].name.replace(".jpg", "")
#     num_digits = len(first_name)
#     
#     # Build correct pattern with starting number
#     if num_digits == 6:
#         pattern = "%06d.jpg"
#     elif num_digits == 5:
#         pattern = "%05d.jpg"
#     else:
#         pattern = "%d.jpg"
#     
#     img_seq_path = str(chin_folder / pattern).replace("\\", "/")
#     print(f"\nUsing pattern: {pattern}")
#     print(f"Full path: {img_seq_path}")
#     print(f"Image count: {len(images)}")
#     
#     # Test ffmpeg with -start_number
#     print(f"\n{'='*60}")
#     print(f"Running ffmpeg...")
#     print(f"{'='*60}")
#     result = subprocess.run([
#         "ffmpeg", 
#         "-start_number", str(first_num),  # ← Start from actual first image number
#         "-framerate", "25", 
#         "-i", img_seq_path,
#         "-c:v", "libx264", "-pix_fmt", "yuv420p", "temp_test.mp4", "-y"
#     ], capture_output=True, text=True)

#     if result.returncode != 0:
#         print("✗ FFMPEG ERROR:")
#         print(result.stderr[-1500:])
#     else:
#         print("✓ Video created successfully!")
#         
#         # Check if temp_test.mp4 was created
#         if Path("temp_test.mp4").exists():
#             file_size_mb = Path('temp_test.mp4').stat().st_size / 1024 / 1024
#             print(f"✓ temp_test.mp4 exists ({file_size_mb:.1f} MB)")
#             
#             # RUN RIFE → 50 FPS
#             print(f"\n{'='*60}")
#             print(f"Running RIFE inference (this will take a few minutes)...")
#             print(f"{'='*60}")
#             result = subprocess.run([
#                 str(RIFE_PY), "inference_video.py",
#                 "--video=temp_test.mp4", "--output=test_50fps.mp4", "--multi=2", "--skip"
#             ], cwd=str(RIFE_DIR), capture_output=True, text=True)

#             if result.returncode != 0:
#                 print("✗ RIFE ERROR:")
#                 if "np.float" in result.stderr:
#                     print("NumPy compatibility issue detected")
#                 print(result.stderr[-1500:])
#             else:
#                 print("✓ RIFE upsampling completed!")
#                 if Path("test_50fps.mp4").exists():
#                     output_size = Path("test_50fps.mp4").stat().st_size / 1024 / 1024
#                     print(f"✓ test_50fps.mp4 created ({output_size:.1f} MB)")
#         else:
#             print("✗ temp_test.mp4 was not created")

# print(f"\n{'='*60}")
# print("✓ Test complete!")
# print(f"{'='*60}")

Found 49 images
First image: 000148.jpg
Last image: 000684.jpg
Starting number: 148

Using pattern: %06d.jpg
Full path: D:/5TH SEM/DL/DL PROJECT/DL_Project_Equine Pain/EQUINE PAIN CODE/dataset/cropped_regions_BALANCED/S1_Video/chin/%06d.jpg
Image count: 49

Running ffmpeg...
✓ Video created successfully!
✓ temp_test.mp4 exists (0.0 MB)

Running RIFE inference (this will take a few minutes)...
✓ RIFE upsampling completed!
✓ test_50fps.mp4 created (0.0 MB)

✓ Test complete!
✓ RIFE upsampling completed!
✓ test_50fps.mp4 created (0.0 MB)

✓ Test complete!


In [ ]:
# ═════════════════════════════════════════════════════════════════════════════
# SETUP — Region and Video Counter
# [DEPRECATED — Superseded by logic in Cell 6 (Main Batch Loop)]
# ═════════════════════════════════════════════════════════════════════════════
# regions = ["chin", "eye_left", "eye_right"]
# 
# video_dirs = [p for p in CROPS_ROOT.iterdir() if p.is_dir()]
# total_videos = len(video_dirs)
# completed = 0

In [54]:
# ==================== MAIN BATCH LOOP ====================
# Process video sequences: convert images → video → RIFE → extract frames
import time

FFMPEG_BIN = str(BASE / "ffmpeg" / "bin" / "ffmpeg.exe")

# Reset counter to 0 for this run
completed = 0

print(f"\n{'='*90}")
print(f"STARTING BATCH PROCESSING: {total_videos} videos × 3 regions")
print(f"Estimated time: 3-5 hours ({total_videos} videos × 15-25 min each)")
print(f"{'='*90}\n")

start_time = time.time()

for video_dir in sorted(video_dirs):
    video_name = video_dir.name
    print(f"\n[{completed + 1}/{total_videos}] Processing: {video_name}")
    print("-" * 70)

    # Create output structure
    output_video_dir = OUTPUT_50FPS / video_name
    output_video_dir.mkdir(parents=True, exist_ok=True)

    all_regions_exist = True

    for region in regions:
        input_folder = video_dir / region
        output_folder = output_video_dir / region
        output_folder.mkdir(parents=True, exist_ok=True)

        # Check if region exists
        if not input_folder.exists():
            print(f"  ⚠ {region:12} → MISSING folder")
            all_regions_exist = False
            continue
        
        images = sorted(list(input_folder.glob("*.jpg")))
        image_count = len(images)
        
        if image_count == 0:
            print(f"  ⚠ {region:12} → NO IMAGES (skipping)")
            all_regions_exist = False
            continue

        # Extract starting frame number for ffmpeg
        first_image_name = images[0].name.replace(".jpg", "")
        first_frame_num = int(first_image_name)
        
        # Determine image pattern (e.g., %06d.jpg)
        num_digits = len(first_image_name)
        if num_digits == 6:
            pattern = "%06d.jpg"
        elif num_digits == 5:
            pattern = "%05d.jpg"
        else:
            pattern = "%d.jpg"

        print(f"  → {region:12} ({image_count:3d} images)...", end=" ", flush=True)

        # Step 1: Convert images to video using ffmpeg
        # CRITICAL FIX: Images may not be consecutive (e.g., 148, 156, 160...)
        # So we create a concat file listing all actual image paths instead of using pattern matching
        temp_video = str(RIFE_ROOT / f"temp_{video_name}_{region}.mp4")
        
        # Create concat demuxer file with explicit list of all images
        concat_file = str(RIFE_ROOT / f"concat_{video_name}_{region}.txt")
        with open(concat_file, "w") as f:
            for img in images:
                img_path = str(img).replace("\\", "/")
                f.write(f"file '{img_path}'\n")
        
        ffmpeg_cmd = [
            FFMPEG_BIN,
            "-f", "concat",
            "-safe", "0",
            "-i", concat_file,
            "-framerate", "25",
            "-c:v", "libx264",
            "-pix_fmt", "yuv420p",
            temp_video, "-y"
        ]
        
        result = subprocess.run(ffmpeg_cmd, capture_output=True, text=True, timeout=300)
        
        if result.returncode != 0 or not Path(temp_video).exists():
            print(f"✗ FFMPEG FAILED")
            print(f"    {result.stderr[-500:]}")
            continue
        
        # DEBUG: Check temp video frame count
        temp_video_size = Path(temp_video).stat().st_size / 1024 / 1024
        print(f"[DEBUG] Temp video: {temp_video_size:.1f} MB", end=" | ")

        # Step 2: Run RIFE upsampling on the video
        output_video = str(RIFE_ROOT / f"output_{video_name}_{region}.mp4")
        
        rife_cmd = [
            str(RIFE_PY),
            str(RIFE_ROOT / "inference_video.py"),
            f"--video={temp_video}",
            f"--output={output_video}",
            "--multi=2",
            "--skip"
        ]
        
        result = subprocess.run(rife_cmd, cwd=str(RIFE_ROOT), capture_output=True, text=True, timeout=3600, errors='replace')
        
        if result.returncode != 0 or not Path(output_video).exists():
            print(f"✗ RIFE FAILED")
            print(f"    STDERR: {result.stderr[-500:]}")
            print(f"    STDOUT: {result.stdout[-500:]}")
            # Cleanup temp video
            if Path(temp_video).exists():
                os.remove(temp_video)
            continue
        
        # DEBUG: Check output video frame count
        output_video_size = Path(output_video).stat().st_size / 1024 / 1024
        probe_cmd = [FFMPEG_BIN, "-v", "error", "-select_streams", "v:0", "-count_packets", "-show_entries", "stream=nb_packets", "-of", "csv=p=0", output_video]
        probe_result = subprocess.run(probe_cmd, capture_output=True, text=True, timeout=30)
        rife_output_frames = probe_result.stdout.strip() if probe_result.returncode == 0 else "?"
        # If probe fails, estimate from file size (rough: ~1MB per 30 frames for H264)
        if rife_output_frames == "?":
            rife_output_frames = max(1, int(output_video_size * 30))  # Rough estimate

        # Step 3: Extract frames from output video using ffmpeg
        # CRITICAL: Use -vsync 0 to extract ALL frames, not just keyframes
        output_pattern = str(output_folder / "%06d.jpg")
        
        extract_cmd = [
            FFMPEG_BIN,
            "-i", output_video,
            "-vsync", "0",           # ← Extract ALL frames, not just keyframes
            output_pattern, "-y"
        ]
        
        result = subprocess.run(extract_cmd, capture_output=True, text=True, timeout=600)
        
        if result.returncode == 0:
            output_images = sorted(list(output_folder.glob("*.jpg")))
            output_count = len(output_images)
            print(f"✓ {output_count} frames (2× upsampled from {image_count})")
        else:
            print(f"✗ EXTRACTION FAILED")
            print(f"    {result.stderr[-200:]}")

        # Cleanup temporary files
        try:
            if Path(temp_video).exists():
                os.remove(temp_video)
            if Path(output_video).exists():
                os.remove(output_video)
            if Path(concat_file).exists():
                os.remove(concat_file)
        except:
            pass

    if all_regions_exist:
        print(f"\n  ✓ {video_name} COMPLETE!")
    else:
        print(f"\n  ⚠ {video_name} PARTIAL (some regions failed)")

    completed += 1
    elapsed = time.time() - start_time
    avg_time = elapsed / completed
    remaining = (total_videos - completed) * avg_time
    print(f"  Progress: {completed}/{total_videos} | Elapsed: {elapsed/60:.1f}m | Est. remaining: {remaining/60:.1f}m")

print("\n" + "=" * 90)
print("✓ BATCH PROCESSING COMPLETE!")
total_time = time.time() - start_time
print(f"  Total time: {total_time/3600:.1f} hours")
print(f"  All 50 FPS upsampled crops saved to: {OUTPUT_50FPS}")
print("=" * 90)


STARTING BATCH PROCESSING: 12 videos × 3 regions
Estimated time: 3-5 hours (12 videos × 15-25 min each)


[1/12] Processing: S10_Video
----------------------------------------------------------------------
  → chin         (127 images)... [DEBUG] Temp video: 0.1 MB | [DEBUG] Temp video: 0.1 MB | ✓ 253 frames (2× upsampled from 127)
  → eye_left     (188 images)... ✓ 253 frames (2× upsampled from 127)
  → eye_left     (188 images)... [DEBUG] Temp video: 0.1 MB | [DEBUG] Temp video: 0.1 MB | ✓ 375 frames (2× upsampled from 188)
  → eye_right    (167 images)... ✓ 375 frames (2× upsampled from 188)
  → eye_right    (167 images)... [DEBUG] Temp video: 0.1 MB | [DEBUG] Temp video: 0.1 MB | ✓ 333 frames (2× upsampled from 167)

  ✓ S10_Video COMPLETE!
  Progress: 1/12 | Elapsed: 2.1m | Est. remaining: 23.0m

[2/12] Processing: S11_Video
----------------------------------------------------------------------
  → chin         (119 images)... ✓ 333 frames (2× upsampled from 167)

  ✓ S10_Video C

In [56]:
# ═════════════════════════════════════════════════════════════════════════════
# STEP 3: VERIFY OUTPUT (Run this after main processing)
# ═════════════════════════════════════════════════════════════════════════════
# Checks that all regions were processed correctly (output_count == 2*input_count - 1)
# Run this cell after completing the main batch processing to identify any failures.

from pathlib import Path

CROP = Path(r"D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\dataset\cropped_regions_BALANCED")
OUT  = Path(r"D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\dataset\cropped_regions_50FPS")

mismatches = []
for video in sorted(CROP.iterdir()):
    for region in ("chin", "eye_left", "eye_right"):
        inp = sorted((video/region).glob("*.jpg"))
        out = sorted((OUT/video.name/region).glob("*.jpg"))
        if len(inp) == 0 and len(out) == 0:
            continue
        # RIFE interpolation: 2*N - 1 frames (inserts frames between existing frames)
        # Example: [A, B, C] → [A, interp(A,B), B, interp(B,C), C] = 5 frames = 2*3-1
        expected = 2 * len(inp) - 1 if len(inp) > 0 else 0
        if len(out) != expected:
            mismatches.append((video.name, region, len(inp), len(out), expected))

print("=" * 80)
print("VERIFICATION RESULTS")
print("=" * 80)
print(f"Mismatches found: {len(mismatches)}")
if mismatches:
    print("\nREGIONS WITH INCORRECT FRAME COUNTS:")
    print("(video, region, input_count, output_count, expected)")
    for m in mismatches:
        print(f"  {m}")
    print("\nTo fix: Run the Focused Reprocessor cell below")
else:
    print("✓ ALL REGIONS VERIFIED — Frame counts correct!")
    print("  Dataset is ready for training/analysis")

VERIFICATION RESULTS
Mismatches found: 0
✓ ALL REGIONS VERIFIED — Frame counts correct!
  Dataset is ready for training/analysis


In [ ]:
# ═════════════════════════════════════════════════════════════════════════════
# DIAGNOSTIC: Test RIFE with full output capture
# [DEPRECATED — Use this only for RIFE troubleshooting, not for main pipeline]
# ═════════════════════════════════════════════════════════════════════════════
# import subprocess
# from pathlib import Path

# BASE = Path(r"D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE")
# RIFE_ROOT = BASE / "Practical-RIFE"
# RIFE_PY = BASE / "rife_env" / "Scripts" / "python.exe"
# CROPS_ROOT = BASE / "dataset" / "cropped_regions_BALANCED"

# # Use S1_Video/chin as test (smallest)
# test_region = CROPS_ROOT / "S1_Video" / "chin"
# images = sorted(list(test_region.glob("*.jpg")))
# print(f"Test folder: {test_region}")
# print(f"Images found: {len(images)}")

# # Create temp video
# first_num = int(images[0].name.replace(".jpg", ""))
# pattern = "%06d.jpg"
# img_seq_path = str(test_region / pattern).replace("\\", "/")

# temp_video = str(RIFE_ROOT / "diag_temp.mp4")
# FFMPEG_BIN = str(BASE / "ffmpeg" / "bin" / "ffmpeg.exe")

# print(f"\n[Step 1] Creating temp video...")
# print(f"  Pattern: {pattern}")
# print(f"  First num: {first_num}")
# print(f"  Image seq: {img_seq_path}")

# result = subprocess.run([
#     FFMPEG_BIN, "-start_number", str(first_num),
#     "-framerate", "25", "-i", img_seq_path,
#     "-c:v", "libx264", "-pix_fmt", "yuv420p",
#     temp_video, "-y"
# ], capture_output=True, text=True)

# if result.returncode == 0 and Path(temp_video).exists():
#     size_mb = Path(temp_video).stat().st_size / 1024 / 1024
#     print(f"  ✓ Created temp video ({size_mb:.1f} MB)")
# else:
#     print(f"  ✗ FFMPEG failed: {result.stderr[-300:]}")
#     temp_video = None

# if temp_video:
#     print(f"\n[Step 2] Running RIFE inference_video.py with FULL output...")
#     print(f"  Temp video: {temp_video}")
#     print(f"  RIFE script: {RIFE_ROOT / 'inference_video.py'}")
#     print(f"  Python: {RIFE_PY}")
#     print("=" * 70)
#     
#     result = subprocess.run([
#         str(RIFE_PY),
#         str(RIFE_ROOT / "inference_video.py"),
#         f"--video={temp_video}",
#         f"--output=diag_output.mp4",
#         "--multi=2",
#         "--skip"
#     ], cwd=str(RIFE_ROOT), capture_output=True, text=True, timeout=300)
#     
#     print("STDOUT:")
#     print(result.stdout if result.stdout else "[EMPTY]")
#     print("\nSTDERR:")
#     print(result.stderr if result.stderr else "[EMPTY]")
#     print(f"\nReturn code: {result.returncode}")
#     print("=" * 70)
#     
#     if Path(RIFE_ROOT / "diag_output.mp4").exists():
#         output_size = (RIFE_ROOT / "diag_output.mp4").stat().st_size / 1024 / 1024
#         print(f"\n✓ Output video created ({output_size:.1f} MB)")
#         
#         # Check frame count
#         print(f"\n[Step 3] Extracting frames...")
#         extract_result = subprocess.run([
#             FFMPEG_BIN, "-i", str(RIFE_ROOT / "diag_output.mp4"),
#             "-vsync", "0", "diag_frame_%06d.jpg", "-y"
#         ], cwd=str(RIFE_ROOT), capture_output=True, text=True)
#         
#         extracted = sorted(list((RIFE_ROOT / ".").glob("diag_frame_*.jpg")))
#         print(f"  Extracted {len(extracted)} frames")
#         if len(extracted) > 0:
#             print(f"  First frame: {extracted[0].name}")
#             print(f"  Last frame: {extracted[-1].name}")
#     else:
#         print(f"\n✗ No output video created by RIFE!")
#         print(f"  Check {RIFE_ROOT / 'diag_output.mp4'}")

# print("\n✓ Diagnostic complete!")

Test folder: D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\dataset\cropped_regions_BALANCED\S1_Video\chin
Images found: 49

[Step 1] Creating temp video...
  Pattern: %06d.jpg
  First num: 148
  Image seq: D:/5TH SEM/DL/DL PROJECT/DL_Project_Equine Pain/EQUINE PAIN CODE/dataset/cropped_regions_BALANCED/S1_Video/chin/%06d.jpg
  ✓ Created temp video (0.0 MB)

[Step 2] Running RIFE inference_video.py with FULL output...
  Temp video: D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\Practical-RIFE\diag_temp.mp4
  RIFE script: D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\Practical-RIFE\inference_video.py
  Python: D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\rife_env\Scripts\python.exe
STDOUT:
skip flag is abandoned, please refer to issue #207.
Loaded 3.x/4.x HD model.
D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\Practical-RIFE\diag_temp.mp4, 1.0 frames in total, 25.0FPS to 50.0FPS
The audio will 

In [ ]:
# ═════════════════════════════════════════════════════════════════════════════
# DIAGNOSTIC: Detailed Image Sequence Debug
# [DEPRECATED — Use this only for debugging ffmpeg image reading issues]
# ═════════════════════════════════════════════════════════════════════════════
# import subprocess
# from pathlib import Path

# BASE = Path(r"D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE")
# FFMPEG_BIN = str(BASE / "ffmpeg" / "bin" / "ffmpeg.exe")
# test_region = BASE / "dataset" / "cropped_regions_BALANCED" / "S1_Video" / "chin"

# images = sorted(list(test_region.glob("*.jpg")))
# print(f"Total images in folder: {len(images)}")
# print(f"First 5 images:")
# for i, img in enumerate(images[:5]):
#     print(f"  {i}: {img.name}")
# print(f"Last 5 images:")
# for i, img in enumerate(images[-5:]):
#     print(f"  {i}: {img.name}")

# # Get the starting number
# first_num = int(images[0].name.replace(".jpg", ""))
# last_num = int(images[-1].name.replace(".jpg", ""))
# print(f"\nFrame range: {first_num} → {last_num}")
# print(f"Expected count: {last_num - first_num + 1}")

# # Test 1: Try with pattern matching directly
# print(f"\n{'='*70}")
# print("TEST 1: Using %06d.jpg pattern with -start_number")
# print(f"{'='*70}")

# pattern = "%06d.jpg"
# img_path_template = str(test_region / pattern).replace("\\", "/")
# print(f"Pattern: {img_path_template}")
# print(f"Start number: {first_num}")

# # Use ffmpeg with -v debug to see what files it's trying to read
# result = subprocess.run([
#     FFMPEG_BIN,
#     "-v", "debug",
#     "-start_number", str(first_num),
#     "-framerate", "25",
#     "-i", img_path_template,
#     "-frames:v", "5",  # Only read first 5 frames
#     "-c:v", "libx264",
#     "-pix_fmt", "yuv420p",
#     "test_debug.mp4", "-y"
# ], capture_output=True, text=True, cwd=str(test_region))

# # Look for "Opening" lines in stderr
# print("\nFFmpeg debug output (file opening attempts):")
# for line in result.stderr.split('\n'):
#     if 'Opening' in line or 'found' in line or 'error' in line.lower():
#         print(f"  {line}")

# if result.returncode != 0:
#     print("\nFFmpeg ERROR:")
#     for line in result.stderr.split('\n')[-20:]:
#         if line.strip():
#             print(f"  {line}")

# # Test 2: Try glob pattern to verify files exist
# print(f"\n{'='*70}")
# print("TEST 2: Direct glob pattern verification")
# print(f"{'='*70}")

# for test_num in [first_num, first_num + 1, first_num + 2, last_num - 1, last_num]:
#     test_file = test_region / f"{test_num:06d}.jpg"
#     exists = test_file.exists()
#     status = "✓" if exists else "✗"
#     print(f"  {status} {test_file.name}")

# # Test 3: Create video with explicit frame range
# print(f"\n{'='*70}")
# print("TEST 3: Using explicit frame range (-start_number and -count_packets)")
# print(f"{'='*70}")

# result = subprocess.run([
#     FFMPEG_BIN,
#     "-start_number", str(first_num),
#     "-framerate", "25",
#     "-i", img_path_template,
#     "-c:v", "libx264",
#     "-pix_fmt", "yuv420p",
#     "-vf", "fps=25",  # Explicit fps filter
#     "test_fps_filter.mp4", "-y"
# ], capture_output=True, text=True, cwd=str(test_region))

# if Path(test_region / "test_fps_filter.mp4").exists():
#     size = (test_region / "test_fps_filter.mp4").stat().st_size / 1024
#     print(f"✓ Video created: {size:.1f} KB")
#     
#     # Check frame count
#     probe = subprocess.run([
#         FFMPEG_BIN,
#         "-v", "error",
#         "-select_streams", "v:0",
#         "-count_frames",
#         "-show_entries", "stream=nb_read_frames",
#         "-of", "csv=p=0",
#         "test_fps_filter.mp4"
#     ], capture_output=True, text=True, cwd=str(test_region))
#     print(f"  Probe output: {probe.stdout.strip()}")
# else:
#     print("✗ Video not created")

# print(f"\n{'='*70}")
# print("✓ Debug complete!")
# print(f"{'='*70}")

Total images in folder: 49
First 5 images:
  0: 000148.jpg
  1: 000156.jpg
  2: 000160.jpg
  3: 000164.jpg
  4: 000168.jpg
Last 5 images:
  0: 000668.jpg
  1: 000672.jpg
  2: 000676.jpg
  3: 000680.jpg
  4: 000684.jpg

Frame range: 148 → 684
Expected count: 537

TEST 1: Using %06d.jpg pattern with -start_number
Pattern: D:/5TH SEM/DL/DL PROJECT/DL_Project_Equine Pain/EQUINE PAIN CODE/dataset/cropped_regions_BALANCED/S1_Video/chin/%06d.jpg
Start number: 148

FFmpeg debug output (file opening attempts):
    configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-cairo --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcapt

In [ ]:
# ═════════════════════════════════════════════════════════════════════════════
# DIAGNOSTIC: Focused Reprocessor for failed regions
# [DEPRECATED — Use Cell 11 (Main RIFE Reprocessor) instead for better logging]
# ═════════════════════════════════════════════════════════════════════════════
# import subprocess, os, sys, shutil, traceback
# from pathlib import Path

# BASE = Path(r"D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE")
# RIFE_ROOT = BASE / "Practical-RIFE"
# FFMPEG_BIN = str(BASE / "ffmpeg" / "bin" / "ffmpeg.exe")
# RIFE_PY = str(BASE / "rife_env" / "Scripts" / "python.exe")

# # Paste the mismatches you reported here (or keep for select checks)
# mismatches = [
#     ('S10_Video', 'chin', 127, 253),
#     ('S10_Video', 'eye_left', 188, 375),
#     ('S10_Video', 'eye_right', 167, 333),
#     ('S11_Video', 'chin', 119, 237),
#     ('S11_Video', 'eye_left', 188, 375),
#     ('S11_Video', 'eye_right', 118, 235),
#     ('S12_Video', 'chin', 108, 215),
#     ('S12_Video', 'eye_left', 129, 1),
#     ('S12_Video', 'eye_right', 76, 1),
#     ('S1_Video', 'chin', 49, 1),
#     ('S1_Video', 'eye_left', 179, 1),
#     ('S1_Video', 'eye_right', 161, 1),
#     ('S2_Video', 'chin', 23, 1),
#     ('S2_Video', 'eye_left', 120, 1),
#     ('S2_Video', 'eye_right', 37, 1),
#     ('S3_Video', 'chin', 116, 1),
#     ('S3_Video', 'eye_left', 112, 1),
#     ('S3_Video', 'eye_right', 53, 1),
#     ('S4_Video', 'chin', 177, 1),
#     ('S4_Video', 'eye_left', 90, 1),
#     ('S4_Video', 'eye_right', 33, 1),
#     ('S5_Video', 'chin', 119, 1),
#     ('S5_Video', 'eye_left', 162, 1),
#     ('S5_Video', 'eye_right', 54, 1),
#     ('S6_Video', 'chin', 82, 1),
#     ('S6_Video', 'eye_left', 187, 1),
#     ('S6_Video', 'eye_right', 71, 1),
#     ('S7_Video', 'chin', 184, 1),
#     ('S7_Video', 'eye_left', 184, 1),
#     ('S7_Video', 'eye_right', 184, 1),
#     ('S8_Video', 'chin', 185, 1),
#     ('S8_Video', 'eye_left', 185, 1),
#     ('S8_Video', 'eye_right', 178, 1),
#     ('S9_Video', 'chin', 42, 1),
#     ('S9_Video', 'eye_left', 83, 1),
#     ('S9_Video', 'eye_right', 57, 1),
# ]

# # Controls
# RUN_RIFE = False   # Set to True to run RIFE for each region (slow)
# CHECK_ONLY = True  # If True, only build temp videos + logs (fast); if False, also extract frames after RIFE

# log_dir = RIFE_ROOT / 'diag_logs'
# log_dir.mkdir(exist_ok=True)

# results = []
# for video_name, region, inp_count, out_count in mismatches:
#     try:
#         print('\n' + '='*60)
#         print(f'Checking: {video_name} / {region}  (in={inp_count} out={out_count})')
#         input_folder = BASE / 'dataset' / 'cropped_regions_BALANCED' / video_name / region
#         output_folder = BASE / 'dataset' / 'cropped_regions_50FPS' / video_name / region
#         output_folder.mkdir(parents=True, exist_ok=True)

#         images = sorted(list(input_folder.glob('*.jpg')))
#         print('  Image files on disk:', len(images))
#         if not images:
#             results.append((video_name, region, 'NO_IMAGES'))
#             continue

#         concat_file = RIFE_ROOT / f'concat_{video_name}_{region}.txt'
#         temp_video = RIFE_ROOT / f'temp_{video_name}_{region}.mp4'
#         output_video = RIFE_ROOT / f'output_{video_name}_{region}.mp4'

#         # create concat file
#         with open(concat_file, 'w', encoding='utf-8') as f:
#             for img in images:
#                 f.write("file '{}'\n".format(str(img).replace('\\', '/')))
#         print('  Created concat file:', concat_file.name)

#         # run ffmpeg to create temp video
#         ff_cmd = [FFMPEG_BIN, '-f', 'concat', '-safe', '0', '-i', str(concat_file), '-framerate', '25', '-c:v', 'libx264', '-pix_fmt', 'yuv420p', str(temp_video), '-y']
#         print('  Running ffmpeg -> temp video...')
#         p = subprocess.run(ff_cmd, capture_output=True, text=True)
#         ff_log = log_dir / f'ffmpeg_create_{video_name}_{region}.log'
#         ff_log.write_text(p.stdout + '\n\nSTDERR:\n' + p.stderr, encoding='utf-8')
#         print('  ffmpeg rc=', p.returncode, 'log->', ff_log.name)
#         if p.returncode != 0 or not temp_video.exists():
#             results.append((video_name, region, 'FFMPEG_FAIL'))
#             continue
#         else:
#             size_mb = temp_video.stat().st_size / 1024 / 1024
#             print(f'  Temp video created ({size_mb:.2f} MB)')

#         if RUN_RIFE:
#             rife_cmd = [RIFE_PY, str(RIFE_ROOT / 'inference_video.py'), f'--video={temp_video}', f'--output={output_video}', '--multi=2', '--skip']
#             print('  Running RIFE (this may take several minutes)...')
#             p = subprocess.run(rife_cmd, cwd=str(RIFE_ROOT), capture_output=True, text=True, errors='replace', timeout=7200)
#             rife_log = log_dir / f'rife_{video_name}_{region}.log'
#             rife_log.write_text('STDOUT:\n' + p.stdout + '\n\nSTDERR:\n' + p.stderr, encoding='utf-8')
#             print('  RIFE rc=', p.returncode, 'log->', rife_log.name)
#             if p.returncode != 0 or not output_video.exists():
#                 results.append((video_name, region, 'RIFE_FAIL'))
#                 continue
#             else:
#                 print('  RIFE output created:', output_video.name)

#             if not CHECK_ONLY:
#                 # extract frames
#                 extract_pattern = str(output_folder / '%06d.jpg')
#                 extract_cmd = [FFMPEG_BIN, '-i', str(output_video), '-vsync', '0', extract_pattern, '-y']
#                 p = subprocess.run(extract_cmd, capture_output=True, text=True)
#                 ext_log = log_dir / f'ffmpeg_extract_{video_name}_{region}.log'
#                 ext_log.write_text(p.stdout + '\n\nSTDERR:\n' + p.stderr, encoding='utf-8')
#                 print('  Extract rc=', p.returncode, 'log->', ext_log.name)
#                 out_count_now = len(sorted(list(output_folder.glob('*.jpg'))))
#                 print('  Output frames now:', out_count_now)
#                 results.append((video_name, region, 'OK_REPROCESSED', out_count_now))
#         else:
#             # Not running RIFE now — report temp video success and existing output count
#             out_existing = len(sorted(list(output_folder.glob('*.jpg'))))
#             results.append((video_name, region, 'TEMP_OK', temp_video.exists(), out_existing))

#     except Exception as e:
#         tb = traceback.format_exc()
#         errlog = log_dir / f'error_{video_name}_{region}.log'
#         errlog.write_text(tb, encoding='utf-8')
#         print('  EXCEPTION ->', errlog.name)
#         results.append((video_name, region, 'EXCEPTION'))

# print('\n' + '='*60)
# print('Summary:')
# for r in results:
#     print(' ', r)
# print('\nLogs saved to', log_dir)
# print("\nNext: If temp videos are OK, set RUN_RIFE=True and re-run the cell for a failing region to produce logs for RIFE.")


Checking: S10_Video / chin  (in=127 out=253)
  Image files on disk: 127
  Created concat file: concat_S10_Video_chin.txt
  Running ffmpeg -> temp video...
  ffmpeg rc= 0 log-> ffmpeg_create_S10_Video_chin.log
  Temp video created (0.10 MB)

Checking: S10_Video / eye_left  (in=188 out=375)
  Image files on disk: 188
  Created concat file: concat_S10_Video_eye_left.txt
  Running ffmpeg -> temp video...
  ffmpeg rc= 0 log-> ffmpeg_create_S10_Video_chin.log
  Temp video created (0.10 MB)

Checking: S10_Video / eye_left  (in=188 out=375)
  Image files on disk: 188
  Created concat file: concat_S10_Video_eye_left.txt
  Running ffmpeg -> temp video...
  ffmpeg rc= 0 log-> ffmpeg_create_S10_Video_eye_left.log
  Temp video created (0.15 MB)

Checking: S10_Video / eye_right  (in=167 out=333)
  Image files on disk: 167
  Created concat file: concat_S10_Video_eye_right.txt
  Running ffmpeg -> temp video...
  ffmpeg rc= 0 log-> ffmpeg_create_S10_Video_eye_left.log
  Temp video created (0.15 MB)

C

In [ ]:
# ═════════════════════════════════════════════════════════════════════════════
# MAIN: RIFE REPROCESSOR (Run only if verification shows mismatches)
# ═════════════════════════════════════════════════════════════════════════════
# This cell reprocesses regions that failed initial upsampling.
# It: (1) rebuilds missing temp MP4s, (2) runs RIFE, (3) extracts frames, (4) verifies counts
# 
# ONLY UNCOMMENT AND RUN THIS IF VERIFICATION CELL SHOWS MISMATCHES!
# SETTINGS:
#   TEST_MODE: True = process only first candidate (safer for debugging)
#   CLEANUP_TEMP: True = delete intermediate MP4 files after success (saves space)
# 
# EXECUTION TIME: ~15-25 minutes per region (depends on frame count)
# ═════════════════════════════════════════════════════════════════════════════

# import subprocess, os, sys, traceback
# from pathlib import Path

# BASE = Path(r"D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE")
# RIFE_ROOT = BASE / "Practical-RIFE"
# FFMPEG_BIN = str(BASE / "ffmpeg" / "bin" / "ffmpeg.exe")
# RIFE_PY = str(BASE / "rife_env" / "Scripts" / "python.exe")

# # ═══════════════════════════════════════════════════════════════════════════
# # CONFIGURATION
# # ═══════════════════════════════════════════════════════════════════════════
# TEST_MODE = False       # True = process only first failing region (for testing)
# CLEANUP_TEMP = True     # True = delete temp MP4s after success (recommended)

# # ═══════════════════════════════════════════════════════════════════════════
# # BUILD CANDIDATE LIST (regions with missing/incorrect output)
# # ═══════════════════════════════════════════════════════════════════════════
# CROPS_ROOT = BASE / 'dataset' / 'cropped_regions_BALANCED'
# OUT_ROOT = BASE / 'dataset' / 'cropped_regions_50FPS'

# candidates = []
# for video_dir in sorted(CROPS_ROOT.iterdir()):
#     for region in ('chin', 'eye_left', 'eye_right'):
#         inp = sorted((video_dir/region).glob('*.jpg'))
#         if len(inp) == 0:
#             continue
#         expected = 2*len(inp) - 1
#         out_dir = OUT_ROOT / video_dir.name / region
#         out_count = len(list(out_dir.glob('*.jpg'))) if out_dir.exists() else 0
#         if out_count != expected:
#             temp_video = RIFE_ROOT / f'temp_{video_dir.name}_{region}.mp4'
#             concat_file = RIFE_ROOT / f'concat_{video_dir.name}_{region}.txt'
#             candidates.append((video_dir.name, region, len(inp), out_count, expected, temp_video, concat_file))

# if not candidates:
#     print('✓ No candidates found — all regions already processed correctly!')
# else:
#     print(f'Found {len(candidates)} regions to reprocess')
#     print(f'Sample: {candidates[:2]}')

# # ═══════════════════════════════════════════════════════════════════════════
# # REPROCESSING LOOP
# # ═══════════════════════════════════════════════════════════════════════════
# log_dir = RIFE_ROOT / 'diag_logs'
# log_dir.mkdir(exist_ok=True)
# processed = []

# print(f'\n{"="*70}')
# print(f'STARTING RIFE REPROCESSING')
# print(f'MODE: {"TEST (1 region only)" if TEST_MODE else "FULL (all candidates)"}')
# print(f'CLEANUP_TEMP: {CLEANUP_TEMP}')
# print(f'Candidates: {len(candidates)}')
# print(f'{"="*70}')

# for idx, (video_name, region, inp_count, out_count, expected, temp_video, concat_file) in enumerate(candidates):
#     if TEST_MODE and idx > 0:
#         print(f'\n*** TEST_MODE: stopping after first candidate ***')
#         break
#     
#     run_idx = idx + 1
#     print(f'\n[{run_idx}/{len(candidates)}] Processing: {video_name} / {region}')
#     print(f'    Input: {inp_count} frames → Expected output: {expected} frames (2*N-1)')

#     # ─────────────────────────────────────────────────────────────────────────
#     # STEP 1: Ensure temp video exists (rebuild from images if needed)
#     # ─────────────────────────────────────────────────────────────────────────
#     if not temp_video.exists():
#         print(f'    → Temp video missing — rebuilding...')
#         if not concat_file.exists():
#             print(f'      → Concat file missing — creating from images...')
#             input_folder = CROPS_ROOT / video_name / region
#             imgs = sorted(list(input_folder.glob('*.jpg')))
#             if not imgs:
#                 print(f'      ✗ No images found — skipping')
#                 processed.append((video_name, region, 'NO_IMAGES'))
#                 continue
#             with open(concat_file, 'w', encoding='utf-8') as f:
#                 for im in imgs:
#                     f.write("file '{}'\n".format(str(im).replace('\\', '/')))
#             print(f'      ✓ Created concat file ({len(imgs)} images)')

#         ff_cmd = [FFMPEG_BIN, '-f', 'concat', '-safe', '0', '-i', str(concat_file), 
#                   '-framerate', '25', '-c:v', 'libx264', '-pix_fmt', 'yuv420p', str(temp_video), '-y']
#         print(f'      → Running ffmpeg (concat → temp MP4)...')
#         p = subprocess.run(ff_cmd, capture_output=True, text=True)
#         create_log = log_dir / f'ffmpeg_create_rebuild_{video_name}_{region}.log'
#         create_log.write_text(p.stdout + '\n\nSTDERR:\n' + p.stderr, encoding='utf-8')
#         if p.returncode != 0 or not temp_video.exists():
#             print(f'      ✗ ffmpeg failed (rc={p.returncode})')
#             processed.append((video_name, region, 'NO_TEMP'))
#             continue
#         else:
#             size_mb = temp_video.stat().st_size / 1024 / 1024
#             print(f'      ✓ Temp video created ({size_mb:.2f} MB)')

#     # ─────────────────────────────────────────────────────────────────────────
#     # STEP 2: Run RIFE upsampling
#     # ─────────────────────────────────────────────────────────────────────────
#     output_video = RIFE_ROOT / f'output_{video_name}_{region}.mp4'
#     rife_log = log_dir / f'reprocess_rife_{video_name}_{region}.log'
#     rife_cmd = [RIFE_PY, str(RIFE_ROOT / 'inference_video.py'), 
#                 f'--video={temp_video}', f'--output={output_video}', '--multi=2', '--skip']
#     print(f'    → Running RIFE (upsampling to 50 FPS)...')
#     try:
#         p_rife = subprocess.run(rife_cmd, cwd=str(RIFE_ROOT), capture_output=True, text=True, 
#                                errors='replace', timeout=7200)
#         rife_log.write_text('STDOUT:\n' + p_rife.stdout + '\n\nSTDERR:\n' + p_rife.stderr, encoding='utf-8')
#         if p_rife.returncode != 0 or not output_video.exists():
#             print(f'      ✗ RIFE failed (rc={p_rife.returncode})')
#             processed.append((video_name, region, 'RIFE_FAIL'))
#             continue
#         else:
#             out_size_mb = output_video.stat().st_size / 1024 / 1024
#             print(f'      ✓ RIFE output created ({out_size_mb:.2f} MB)')
#     except Exception as e:
#         tb = traceback.format_exc()
#         rife_log.write_text(tb, encoding='utf-8')
#         print(f'      ✗ RIFE exception')
#         processed.append((video_name, region, 'RIFE_EXCEPTION'))
#         continue

#     # ─────────────────────────────────────────────────────────────────────────
#     # STEP 3: Extract frames from upsampled video
#     # ─────────────────────────────────────────────────────────────────────────
#     out_folder = OUT_ROOT / video_name / region
#     out_folder.mkdir(parents=True, exist_ok=True)
#     extract_pattern = str(out_folder / '%06d.jpg')
#     extract_cmd = [FFMPEG_BIN, '-i', str(output_video), '-vsync', '0', extract_pattern, '-y']
#     print(f'    → Extracting frames...')
#     p_extract = subprocess.run(extract_cmd, capture_output=True, text=True)
#     extract_log = log_dir / f'ffmpeg_extract_{video_name}_{region}.log'
#     extract_log.write_text(p_extract.stdout + '\n\nSTDERR:\n' + p_extract.stderr, encoding='utf-8')
#     
#     out_count_now = len(sorted(list(out_folder.glob('*.jpg'))))
#     print(f'      Output frames: {out_count_now} (expected {expected})')

#     # ─────────────────────────────────────────────────────────────────────────
#     # STEP 4: Validate and cleanup
#     # ─────────────────────────────────────────────────────────────────────────
#     if p_rife.returncode == 0 and p_extract.returncode == 0 and out_count_now == expected:
#         print(f'    ✓ SUCCESS — all frames correct')
#         processed.append((video_name, region, 'OK', out_count_now))
#         if CLEANUP_TEMP:
#             try:
#                 if temp_video.exists():
#                     temp_video.unlink()
#                 if output_video.exists():
#                     output_video.unlink()
#                 print(f'      Cleaned up temp files')
#             except Exception as e:
#                 print(f'      Warning: cleanup error')
#     else:
#         if out_count_now != expected:
#             print(f'    ✗ MISMATCH — {out_count_now} ≠ {expected}')
#         else:
#             print(f'    ✗ ERROR — RIFE or extraction failed')
#         processed.append((video_name, region, 'EXTRACTION_MISMATCH', out_count_now))

# # ═══════════════════════════════════════════════════════════════════════════
# # SUMMARY
# # ═══════════════════════════════════════════════════════════════════════════
# print(f'\n{"="*70}')
# print(f'REPROCESSING COMPLETE')
# print(f'{"="*70}')
# ok_count = sum(1 for r in processed if len(r) >= 3 and r[2] == 'OK')
# fail_count = len(processed) - ok_count
# print(f'  ✓ OK:        {ok_count}/{len(processed)}')
# print(f'  ✗ FAILED:    {fail_count}/{len(processed)}')
# print(f'\nNext: Run the Verification cell (Step 3) to confirm all regions are now correct.')
# print(f'Logs: {log_dir}')

No candidates found (nothing to reprocess).

STARTING REPROCESSING RUN
MODE: FULL (all candidates)
CLEANUP_TEMP: True
Candidates to process: 0

REPROCESSING COMPLETE

Results Summary:
  ✓ OK:        0
  ✗ FAILED:    0

Detailed results:

Logs location: D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\Practical-RIFE\diag_logs
Re-run verification cell to confirm all regions now match expected frame counts.
